In [25]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

# questions:
- we have 2 dataset one for user and one for order

In [41]:
DATA = Path("../data/processed/analytics_table.parquet")
FIGS = Path("../reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)


DEFAULT_RAW_DIR = Path("../data/raw")
DEFAULT_RAW_DIR.mkdir(parents=True, exist_ok=True)


In [46]:
import sys
sys.path.append("../")

from python_scripts import run_day1_load,run_day2_clean,run_day3_build_analytics


run_day1_load.main()
run_day2_clean.main()
run_day3_build_analytics.main()


INFO python_scripts.run_day1_load: Loaded rows: orders=5250 users=1000
INFO python_scripts.run_day1_load: Orders dtypes:
order_id       object
user_id        object
amount        float64
quantity        Int64
created_at     object
status         object
dtype: object
INFO python_scripts.run_day1_load: Wrote: /home/khaled/SDAIA/week_2/data/processed
INFO python_scripts.run_day1_load: Run meta: /home/khaled/SDAIA/week_2/data/processed/_run_meta.json
INFO python_scripts.run_day2_clean: loading raw inputs
INFO python_scripts.run_day2_clean: rows: order_raw=5250, users=1000
INFO python_scripts.run_day2_clean: wrote missingness report : /home/khaled/SDAIA/week_2/reports/missingness_order.csv
INFO python_scripts.run_day2_clean: wrote processed output: /home/khaled/SDAIA/week_2/data/processed


missing created_at after parse: 507 / 5250
rows: 5250
country match rate: 1.0
wrote: /home/khaled/SDAIA/week_2/data/processed/analytics_table.parquet
  country     n    revenue
0      AE  1348  324843.68
1      KW  1326  307761.92
2      QA  1344  292329.93
3      SA  1232  286976.39


/home/khaled/SDAIA/week_2/src/bootcamp_data/transforms.py:45: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [27]:
def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires `kaleido`)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

In [28]:
df = pd.read_parquet(DATA)

print(f"rows: {len(df)} cols: {len(df.columns)}")
print(df.dtypes.head(15))
print("-"*90)
missing = df.isna().sum().sort_values(ascending=False).head(8)
print(f"top missing columns: {missing}")

rows: 5250 cols: 18
order_id                       object
user_id                        object
amount                        float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
data                           object
year                          float64
month                          object
dow                            object
hour                          float64
country                        object
dtype: object
------------------------------------------------------------------------------------------
top missing columns: quantity        534
status          513
status_clean    513
hour            507
data            507
created_at      507
dow             507
year            507
dtype: int64


In [29]:
df

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,data,year,month,dow,hour,country,signup_date,amount_winsor,amount__is_outlier
0,A4434,0021,479.46,6,2025-10-06 15:00:00+00:00,refunded,refund,False,False,2025-10-06,2025.0,2025-10,Monday,15.0,AE,2025-01-08,479.46,False
1,A0177,0677,124.64,4,2025-06-15 01:35:00+00:00,Refunded,refund,False,False,2025-06-15,2025.0,2025-06,Sunday,1.0,AE,2025-08-29,124.64,False
2,A2972,0763,NaN,4,2025-11-14 23:05:00+00:00,Refunded,refund,True,False,2025-11-14,2025.0,2025-11,Friday,23.0,SA,2025-09-28,NaN,False
3,A2288,0197,397.26,7,2025-10-16 20:04:00+00:00,Paid,paid,False,False,2025-10-16,2025.0,2025-10,Thursday,20.0,QA,2025-03-11,397.26,False
4,A3985,0460,374.25,1,2025-01-16 03:21:00+00:00,Pending,pending,False,False,2025-01-16,2025.0,2025-01,Thursday,3.0,AE,2025-06-13,374.25,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,A4359,0728,325.60,5,2025-07-06 10:15:00+00:00,Refunded,refund,False,False,2025-07-06,2025.0,2025-07,Sunday,10.0,AE,2025-09-16,325.60,False
5246,A2386,0289,90.63,3,2025-12-16 01:16:00+00:00,paid,paid,False,False,2025-12-16,2025.0,2025-12,Tuesday,1.0,AE,2025-04-13,90.63,False
5247,A2828,0228,389.80,3,2025-08-21 15:00:00+00:00,Paid,paid,False,False,2025-08-21,2025.0,2025-08,Thursday,15.0,QA,2025-03-22,389.80,False
5248,A0014,0577,156.48,2,2025-12-19 10:54:00+00:00,refunded,refund,False,False,2025-12-19,2025.0,2025-12,Friday,10.0,SA,2025-07-24,156.48,False


In [30]:
rev = (
    df.groupby("country", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue = ("amount", "sum"),
        aov = ("amount", "mean"),
    )
    .reset_index()
    .sort_values("revenue", ascending=False),
)
rev

(  country     n    revenue         aov
 0      AE  1348  324843.68  257.607994
 1      KW  1326  307761.92  260.153779
 2      QA  1344  292329.93  244.014967
 3      SA  1232  286976.39  257.840422,)

In [31]:
rev = (
    df.groupby("country", dropna=False)
      .agg(
          n=("order_id","size"),
          revenue=("amount","sum"),
          aov=("amount","mean"),
      )
      .reset_index()
      .sort_values("revenue", ascending=False)
)
print(rev)
print(type(rev))


  country     n    revenue         aov
0      AE  1348  324843.68  257.607994
1      KW  1326  307761.92  260.153779
2      QA  1344  292329.93  244.014967
3      SA  1232  286976.39  257.840422
<class 'pandas.core.frame.DataFrame'>


## what is the highest revenue by country

In [32]:
fig = px.bar(rev, x="country", y="revenue", title="Revenue by country (all data)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
save_fig(fig, FIGS / "revenue_by_country.png")
fig

### AE has the most revenue

In [33]:
trend = (
    df.groupby("month", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum"),
    )
    .reset_index()
    .sort_values("month")
)

## what is Revenue trend (monthly)

In [34]:
fig = px.line(trend, x="month", y="revenue", title="Revenue over time (monthly)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue")
save_fig(fig, FIGS / "revenue_trend_monthly.png")
fig

#### after nov 2025 decrease

In [39]:
fig = px.histogram(df, x="amount_winsor", nbins=30, title="Order amount distribution (winsorized)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
save_fig(fig, FIGS / "amount_hist_winsor.png", )
fig

#### Most orders fall between ~50 and ~450

### Q)4 which country has the highest revenue in each months

In [47]:
rev_monthly = (
    df.groupby(["month", "country"], dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum"),
        aov=("amount", "mean"),
    )
    .reset_index()
    .sort_values("month") # Ensures the chart displays months in chronological order
)

In [49]:
fig = px.bar(
    rev_monthly,
    x="month",
    y="revenue",
    color="country",
    title="Revenue by Month and Country",
    barmode="group" # Use "group" for side-by-side or remove for stacked bars
)

fig.update_layout(title={"x": 0.02}, xaxis_tickangle=-45)
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Total Revenue")

save_fig(fig, FIGS / "revenue_by_month.png")
fig.show()

INFO choreographer.browsers.chromium: Chromium init'ed with kwargs {}
INFO choreographer.browsers.chromium: Found chromium path: /usr/bin/google-chrome
INFO choreographer.utils._tmpfile: Temp directory created: /tmp/tmpynu45o1k.
INFO choreographer.browser_async: Opening browser.
INFO choreographer.utils._tmpfile: Temp directory created: /tmp/tmpg78n8f1o.
INFO choreographer.browsers.chromium: ldd failed. e: Command '['ldd', '/usr/bin/google-chrome']' returned non-zero exit status 1., stderr: None
INFO choreographer.browsers.chromium: Temporary directory at: /tmp/tmpg78n8f1o
INFO kaleido.kaleido: Conforming 1 to file:///tmp/tmpynu45o1k/index.html
INFO kaleido.kaleido: Waiting on all navigates
INFO kaleido.kaleido: All navigates done, putting them all in queue.
INFO kaleido.kaleido: Getting tab from queue (has 1)
INFO kaleido.kaleido: Got E7C1
INFO kaleido._kaleido_tab: Processing Revenue_by_Month_and_Country.png
INFO kaleido._kaleido_tab: Sending big command for Revenue_by_Month_and_Coun

#### kw has the most 5 times

In [36]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }

d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))
res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
print(res)

n_SA: 1232 n_AE: 1348
{'diff_mean': 0.020393367759836578, 'ci_low': -0.017703415834906974, 'ci_high': 0.05933074251416237}


sa 2% higher than AE